In [47]:
import os
import re
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

import numpy as np

from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Masking, Dropout
from tensorflow.keras.models import Model

In [48]:
import os
file_path = os.getenv('HOME')+'/aiffel/translator_seq2seq/data/fra.txt'
lines = pd.read_csv(file_path, names=['eng', 'fra', 'cc'], sep='\t')
print('전체 샘플의 수:', len(lines))
lines.sample(5)

전체 샘플의 수: 217975


,eng,fra,cc
192143,I'm afraid that there isn't any coffee left.,Je crains qu'il ne reste plus de café.,CC-BY 2.0 (France) Attribution: tatoeba.org #7...
148697,Do you think that you can beat me?,Pensez-vous que vous puissiez me battre ?,CC-BY 2.0 (France) Attribution: tatoeba.org #3...
181353,"To tell you the truth, I don't love him.","À la vérité, je ne l'aime pas.",CC-BY 2.0 (France) Attribution: tatoeba.org #2...
161901,I promise you won't be disappointed.,Je vous promets que vous ne serez pas déçues.,CC-BY 2.0 (France) Attribution: tatoeba.org #6...
9463,Tom's unarmed.,Tom n’est pas armé.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...


In [49]:
lines = lines[['eng', 'fra']][60000:93000]
lines.sample(5)

,eng,fra
69200,Don't wear yourself out.,Ne t'épuise pas.
66681,Tom won't get involved.,Tom ne sera pas impliqué.
91904,Tom didn't talk about you.,Tom n'a pas parlé de toi.
61129,Don't say a word to me.,Ne me dis pas un mot !
86188,He called on me yesterday.,Il m'a rendu visite hier.


In [50]:
lines_np_eng= lines['eng'].to_numpy()
lines_np_fra= lines['fra'].to_numpy()
lines_np_eng

array(['You were there, right?', 'You will stay at home.',
       "You won't be punished.", ..., 'What time did you wake up?',
       'What time did you wake up?', 'What time did you wake up?'],
      dtype=object)

In [51]:
sos_token = '<start> '
eos_token = ' <end>'

def preprocess_line(line, plus_token = True):
    # 소문자로 변경하기
    line = line.lower().strip()
    # 구두점(Punctuation)을 단어와 분리하기
    line = re.sub(r"([?.!,¿])", r" \1 ", line)
    line = re.sub(r'[" "]+', " ", line)
    line = re.sub(r"[^a-zA-Z?.!,¿]+", " ", line)

    line = line.strip()
    
    if plus_token == True:
        line = sos_token + line + eos_token
    
    return line

In [52]:
def tokenize(corpus):
    tokenizer = Tokenizer(
        num_words=7000,  
        filters=' ',   
        oov_token="<unk>"  
    )
    tokenizer.fit_on_texts(corpus)  

    tensor = tokenizer.texts_to_sequences(corpus)   

    return tensor, tokenizer

In [53]:
eng_lines = []
fra_lines = []

# eng_lines.append(lines.eng.apply(lambda x : preprocess_line(x,plus_token = False)))
# fra_lines.append(lines.fra.apply(lambda x : preprocess_line(x),))

for eng, fra in zip(lines.eng, lines.fra):
    if len(eng) == 0: continue
    if len(fra) == 0: continue   
        
    eng_lines.append(preprocess_line(eng, plus_token = False))
    fra_lines.append(preprocess_line(fra))

In [54]:
np.shape(eng_lines)

(33000,)

In [55]:
eng_tensor, eng_tokenizer = tokenize(eng_lines)
fra_tensor, fra_tokenizer = tokenize(fra_lines)
fra_tensor[:10]

[[2, 15, 105, 23, 374, 34, 19, 7, 22, 8, 6, 3],
 [2, 15, 3036, 151, 78, 4, 3],
 [2, 15, 13, 1135, 8, 3723, 4, 3],
 [2, 10, 19, 347, 8, 16, 1519, 4, 3],
 [2, 15, 13, 389, 8, 16, 1519, 4, 3],
 [2, 10, 19, 1647, 8, 17, 4, 3],
 [2, 15, 443, 176, 9, 5098, 4, 3],
 [2, 15, 443, 176, 9, 35, 20, 128, 278, 4, 3],
 [2, 15, 443, 176, 9, 26, 841, 72, 4, 3],
 [2, 10, 1136, 176, 9, 10, 841, 72, 4, 3]]

In [56]:
encoder_input = eng_tensor
# 종료 토큰 제거
decoder_input = [[char for char in line if char != fra_tokenizer.word_index['<end>']] for line in fra_tensor]
# 시작 토큰 제거
decoder_target =[[char for char in line if char != fra_tokenizer.word_index['<start>']] for line in fra_tensor]

In [58]:
def pad_tensor(tensor):
    total_data_text = list(tensor)
    num_tokens = [len(tokens) for tokens in total_data_text]
    max_tokens = max(num_tokens)
#     max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
    maxlen = int(max_tokens)
    tensor = pad_sequences(tensor, padding='post', maxlen=maxlen)  
    return tensor

In [59]:
encoder_input = pad_tensor(encoder_input)
decoder_input = pad_tensor(decoder_input)
decoder_target = pad_tensor(decoder_target)
print('영어 데이터의 크기(shape) :',np.shape(encoder_input))
print('프랑스어 입력데이터의 크기(shape) :',np.shape(decoder_input))
print('프랑스어 출력데이터의 크기(shape) :',np.shape(decoder_target))

영어 데이터의 크기(shape) : (33000, 10)
프랑스어 입력데이터의 크기(shape) : (33000, 17)
프랑스어 출력데이터의 크기(shape) : (33000, 17)


In [60]:
eng_vocab_size = len(eng_tokenizer.word_index)+1
fra_vocab_size = len(fra_tokenizer.word_index)+1

max_eng_seq_len = encoder_input.shape[1]
max_fra_seq_len = decoder_input.shape[1]

In [61]:
print('전체 샘플의 수 :',len(lines))
print('영어 단어장의 크기 :', eng_vocab_size)
print('프랑스어 단어장의 크기 :', fra_vocab_size)
print('영어 시퀀스의 최대 길이', max_eng_seq_len)
print('프랑스어 시퀀스의 최대 길이', max_fra_seq_len)

전체 샘플의 수 : 33000
영어 단어장의 크기 : 5795
프랑스어 단어장의 크기 : 8297
영어 시퀀스의 최대 길이 10
프랑스어 시퀀스의 최대 길이 17


In [62]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [63]:
n_of_val = 3000

encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

print(encoder_input_train.shape)
print(decoder_input_train.shape)
print(decoder_target_train.shape)
print(encoder_input_test.shape)
print(decoder_input_test.shape)
print(decoder_target_test.shape)

(30000, 10)
(30000, 17)
(30000, 17)
(3000, 10)
(3000, 17)
(3000, 17)


In [64]:
embedding_size = 512
hidden_size = 512
# 인코더에서 사용할 임베딩 층 사용 예시
encoder_inputs = Input(shape=(None, ), name='encoder_input')
enc_emb =  Embedding(eng_vocab_size, embedding_size,
                    input_length=max_eng_seq_len)(encoder_inputs)
enc_masking = Masking(mask_value=0.0)(enc_emb)
encoder_lstm = LSTM(hidden_size, dropout = 0.5, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking)
encoder_states = [state_h, state_c]

In [65]:
decoder_inputs = Input(shape=(None, ), name='decoder_input')
dec_emb =  Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
dec_masking = Masking(mask_value=0.0)(dec_emb)
decoder_lstm = LSTM(hidden_size, dropout = 0.5, return_sequences = True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_masking, initial_state = encoder_states)

In [66]:
decoder_softmax_layer = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [67]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [68]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 512)    2967040     encoder_input[0][0]              
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 512)    4248064     decoder_input[0][0]              
____________________________________________________________________________________________

In [69]:
model.fit(x=[encoder_input_train, decoder_input_train], 
          y=decoder_target_train, 
          validation_data = ([encoder_input_test, decoder_input_test], 
                             decoder_target_test),
          batch_size=32, 
          epochs=50)

Epoch 1/50
938/938 [==============================] - 60s 22ms/step - loss: 1.9338 - val_loss: 1.6347
Epoch 2/50
938/938 [==============================] - 18s 20ms/step - loss: 1.4711 - val_loss: 1.4149
Epoch 3/50
938/938 [==============================] - 18s 19ms/step - loss: 1.2786 - val_loss: 1.2803
Epoch 4/50
938/938 [==============================] - 18s 19ms/step - loss: 1.1427 - val_loss: 1.1875
Epoch 5/50
938/938 [==============================] - 18s 19ms/step - loss: 1.0369 - val_loss: 1.1232
Epoch 6/50
938/938 [==============================] - 18s 19ms/step - loss: 0.9560 - val_loss: 1.0834
Epoch 7/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8877 - val_loss: 1.0500
Epoch 8/50
938/938 [==============================] - 18s 19ms/step - loss: 0.8338 - val_loss: 1.0355
Epoch 9/50
938/938 [==============================] - 18s 19ms/step - loss: 0.7945 - val_loss: 1.0264
Epoch 10/50
938/938 [==============================] - 18s 19ms/step - loss: 0.766

In [70]:
encoder_model = Model(inputs = encoder_inputs, outputs = encoder_states)
encoder_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
encoder_input (InputLayer)   [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 512)         2967040   
_________________________________________________________________
masking_2 (Masking)          (None, None, 512)         0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 512), (None, 512) 2099200   
Total params: 5,066,240
Trainable params: 5,066,240
Non-trainable params: 0
_________________________________________________________________


In [71]:
decoder_state_input_h = Input(shape=(embedding_size,))
decoder_state_input_c = Input(shape=(embedding_size,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2 = Embedding(fra_vocab_size, embedding_size)(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

decoder_outputs2 = decoder_softmax_layer(decoder_outputs2)

In [72]:
eng2idx = eng_tokenizer.word_index
fra2idx = fra_tokenizer.word_index
idx2eng = eng_tokenizer.index_word
idx2fra = fra_tokenizer.index_word

In [73]:
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs2] + decoder_states2)
decoder_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
decoder_input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, None, 512)    4248064     decoder_input[0][0]              
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512)]        0                                            
____________________________________________________________________________________________

In [74]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <start>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1,1))
    target_seq[0, 0] = fra2idx['<start>']
    
    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = idx2fra[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += ' '+sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '<end>' or
           len(decoded_sentence) > max_fra_seq_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [75]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2src(input_seq):
    temp=''
    for i in input_seq:
        if(i!=0):
            temp = temp + idx2eng[i]+' '
    return temp

In [76]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
    temp=''
    for i in input_seq:
        if((i!=0 and i!=fra2idx['<start>']) and i!=fra2idx['<end>']):
            temp = temp + idx2fra[i] + ' '
    return temp

In [77]:
for seq_index in [1,201,501,1004,2015]:
    input_seq = encoder_input_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', seq2src(encoder_input_test[seq_index]))
    print('정답 문장:', seq2tar(decoder_input_test[seq_index]))
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1])

-----------------------------------
입력 문장: i want to go right now . 
정답 문장: je veux y aller tout de suite . 
번역기가 번역한 문장:  je veux me me maintenan
-----------------------------------
입력 문장: i m not swallowing that . 
정답 문장: je ne vais pas avaler a . 
번역기가 번역한 문장:  je ne je je ne . 
-----------------------------------
입력 문장: i don t want to be alone . 
정답 문장: je ne veux pas tre seule . 
번역기가 번역한 문장:  je ne que veux veu
-----------------------------------
입력 문장: i ate a hot dog for lunch . 
정답 문장: j ai mang un hot dog au d jeuner . 
번역기가 번역한 문장:  j j un un un d chie
-----------------------------------
입력 문장: our tv is out of order . 
정답 문장: notre t l viseur est hors service . 
번역기가 번역한 문장:  notre t l l la l
